## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

#sys import due to data file; reference https://csatlas.com/python-import-file-module/#import_a_file_in_a_different_directory
import sys
sys.path.append( '..' )
import config

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,2021-10-24 21:56:56,22.0752,-159.3190,84.18,78,42,4.00,scattered clouds
1,1,Victoria,HK,2021-10-24 21:54:37,22.2855,114.1577,70.25,74,37,1.01,scattered clouds
2,2,Jamestown,US,2021-10-24 21:56:57,42.0970,-79.2353,51.12,97,90,1.99,moderate rain
3,3,Hervey Bay,AU,2021-10-24 21:56:57,-25.2986,152.8535,81.03,82,28,10.09,scattered clouds
4,4,Leh,IN,2021-10-24 21:53:27,34.1667,77.5833,29.16,87,91,4.23,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))        


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,2021-10-24 21:56:56,22.0752,-159.3190,84.18,78,42,4.00,scattered clouds
3,3,Hervey Bay,AU,2021-10-24 21:56:57,-25.2986,152.8535,81.03,82,28,10.09,scattered clouds
6,6,Bauchi,NG,2021-10-24 21:56:58,10.5000,10.0000,77.50,44,94,3.87,overcast clouds
8,8,San Cristobal,VE,2021-10-24 21:56:59,7.7669,-72.2250,75.38,70,67,2.37,broken clouds
11,11,Vanimo,PG,2021-10-24 21:57:00,-2.6741,141.3028,78.60,88,100,4.94,overcast clouds
12,12,Atuona,PF,2021-10-24 21:56:14,-9.8000,-139.0333,77.92,73,2,17.36,clear sky
14,14,Dingle,PH,2021-10-24 21:53:18,10.9995,122.6711,75.47,92,99,2.64,overcast clouds
15,15,Panaba,MX,2021-10-24 21:52:14,21.2833,-88.2667,89.22,57,98,10.67,overcast clouds
20,20,Georgetown,MY,2021-10-24 21:53:04,5.4112,100.3354,77.90,94,20,3.00,few clouds
23,23,Americus,US,2021-10-24 21:57:05,32.0724,-84.2327,75.33,60,100,5.75,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                222
City                   222
Country                222
Date                   222
Lat                    222
Lng                    222
Max Temp               222
Humidity               222
Cloudiness             222
Wind Speed             222
Current Description    222
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,
3,Hervey Bay,AU,81.03,scattered clouds,-25.2986,152.8535,
6,Bauchi,NG,77.50,overcast clouds,10.5000,10.0000,
8,San Cristobal,VE,75.38,broken clouds,7.7669,-72.2250,
11,Vanimo,PG,78.60,overcast clouds,-2.6741,141.3028,
12,Atuona,PF,77.92,clear sky,-9.8000,-139.0333,
14,Dingle,PH,75.47,overcast clouds,10.9995,122.6711,
15,Panaba,MX,89.22,overcast clouds,21.2833,-88.2667,
20,Georgetown,MY,77.90,few clouds,5.4112,100.3354,
23,Americus,US,75.33,overcast clouds,32.0724,-84.2327,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Hervey Bay,AU,81.03,scattered clouds,-25.2986,152.8535,Shelly Bay Resort
6,Bauchi,NG,77.50,overcast clouds,10.5000,10.0000,
8,San Cristobal,VE,75.38,broken clouds,7.7669,-72.2250,Posada Villaven C.A.
11,Vanimo,PG,78.60,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
...,...,...,...,...,...,...,...
690,San Angelo,US,92.10,clear sky,31.4638,-100.4370,"Motel 6 San Angelo, TX"
692,Tiarei,PF,77.02,moderate rain,-17.5333,-149.3333,Le Rocher de Tahiti
694,Sabha,LY,78.75,clear sky,27.0377,14.4283,Foundation Taaj Asala for Historical Research ...
695,Mahibadhoo,MV,82.60,scattered clouds,3.7833,72.9667,Omadhoo Inn


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))